In [3]:
from keras.applications import VGG16

In [28]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [29]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [8]:
original_dataset_dir = 'F:\\study\\ml\\DataSet\\dogs_vs_cats\original'
base_dir = 'F:\study\ml\DataSet\dogs_vs_cats\cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [9]:
datagen = ImageDataGenerator(rescale=1. / 255)
batch_size = 20

In [13]:
def extract_feature(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    lables = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                            target_size=(150, 150),
                                            batch_size=batch_size,
                                            class_mode='binary')
    i = 0
    for input_batch, labels_batch in generator:
        features_batch = conv_base.predict(input_batch)
        features[i * batch_size:(i + 1) * batch_size] = features_batch
        lables[i * batch_size:(i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, lables

In [30]:
train_features, train_labels = extract_feature(train_dir, 2000)
validation_features, validation_labels = extract_feature(validation_dir, 1000)
test_features, test_labels = extract_feature(test_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [31]:
train_features.shape

(2000, 4, 4, 512)

In [32]:
train_features[0]

array([[[0.91105127, 0.        , 0.        , ..., 0.        ,
         1.13704181, 0.        ],
        [1.25691807, 0.        , 0.        , ..., 0.        ,
         0.56305552, 0.        ],
        [0.01481284, 0.        , 0.58635139, ..., 0.        ,
         0.15478727, 0.        ],
        [0.49365985, 0.        , 0.        , ..., 0.        ,
         0.59466094, 0.        ]],

       [[0.80253357, 0.        , 0.        , ..., 0.        ,
         1.03387094, 0.        ],
        [1.1789521 , 0.        , 0.72089756, ..., 0.        ,
         0.75889981, 0.        ],
        [0.56917733, 0.        , 1.07801533, ..., 0.        ,
         0.        , 0.        ],
        [0.53726596, 0.        , 0.32577533, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.71279502, 0.        ],
        [0.13314265, 0.        , 0.31844726, ..., 0.        ,
         0.37937379, 0.        ],
        [0.2722365 , 0.        , 0.4

In [15]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [16]:
from keras import models, layers, optimizers

In [17]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [18]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

In [19]:
history = model.fit(train_features,
                    train_labels,
                    epochs=30,
                    batch_size=20,
                    validation_data=(validation_features, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.6016 - acc: 0.6770 - val_loss: 0.4554 - val_acc: 0.8020
Epoch 2/30
2000/2000 [==============================] - 3s 2ms/step - loss: 0.4248 - acc: 0.8120 - val_loss: 0.3683 - val_acc: 0.8470
Epoch 3/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3556 - acc: 0.8530 - val_loss: 0.3220 - val_acc: 0.8840
Epoch 4/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.3099 - acc: 0.8790 - val_loss: 0.3157 - val_acc: 0.8650
Epoch 5/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2837 - acc: 0.8915 - val_loss: 0.2829 - val_acc: 0.8950
Epoch 6/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2632 - acc: 0.8975 - val_loss: 0.2772 - val_acc: 0.8900
Epoch 7/30
2000/2000 [==============================] - 4s 2ms/step - loss: 0.2453 - acc: 0.9070 - val_loss: 0.2634 - val_acc: 0.8970
Epoch 8/30
200

In [33]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [35]:
conv_base.trainable = False

In [36]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [37]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [38]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [39]:
validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [40]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

In [41]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=validation_generator,
                              validation_steps=50)

Epoch 1/30
100/100 [==============================] - 362s 4s/step - loss: 0.5856 - acc: 0.6980 - val_loss: 0.4477 - val_acc: 0.8250
Epoch 2/30
100/100 [==============================] - 360s 4s/step - loss: 0.4822 - acc: 0.7845 - val_loss: 0.3792 - val_acc: 0.8440
Epoch 3/30
100/100 [==============================] - 359s 4s/step - loss: 0.4252 - acc: 0.8150 - val_loss: 0.3438 - val_acc: 0.8670
Epoch 4/30
100/100 [==============================] - 359s 4s/step - loss: 0.4045 - acc: 0.8230 - val_loss: 0.3133 - val_acc: 0.8760
Epoch 5/30
100/100 [==============================] - 359s 4s/step - loss: 0.3904 - acc: 0.8320 - val_loss: 0.2982 - val_acc: 0.8740
Epoch 6/30
100/100 [==============================] - 359s 4s/step - loss: 0.3704 - acc: 0.8330 - val_loss: 0.2839 - val_acc: 0.8850
Epoch 7/30
100/100 [==============================] - 359s 4s/step - loss: 0.3614 - acc: 0.8380 - val_loss: 0.2747 - val_acc: 0.8900
Epoch 8/30
100/100 [==============================] - 359s 4s/step - 

fine-tuning

In [42]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [43]:
conv_base.trainable = True

In [45]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable == True:
        layer.trainable = True
    else:
        layer.trainable = False

In [46]:
for l in conv_base.layers:
    print(str(l) + '--' + str(l.trainable))

<keras.engine.input_layer.InputLayer object at 0x000000001D64F748>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D64FF28>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D64FDD8>--False
<keras.layers.pooling.MaxPooling2D object at 0x000000001D65C0F0>--False
<keras.layers.convolutional.Conv2D object at 0x000000001E9EF048>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D62DCC0>--False
<keras.layers.pooling.MaxPooling2D object at 0x000000001D61EE48>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D635C88>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D616F98>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D6186D8>--False
<keras.layers.pooling.MaxPooling2D object at 0x000000001B8D6898>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D622978>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D60E358>--False
<keras.layers.convolutional.Conv2D object at 0x000000001D63971

In [47]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [48]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=100,
                              validation_data=validation_generator,
                              validation_steps=50)

Epoch 1/100
  5/100 [>.............................] - ETA: 5:30 - loss: 0.4088 - acc: 0.7900

KeyboardInterrupt: 